# SnowFlake Right Sizing-  Workflows in Control-M using Python

# Step 1 - Setup

## Step 1A - Install the library

In [1]:
import ctm_python_client as ctm

## Step 1B- Import the library

In [2]:
from ctm_python_client.core.bmc_control_m import CmJobFlow
from ctm_python_client.jobs.dummy import DummyJob
from ctm_python_client.jobs.command import CommandJob 
from ctm_python_client.jobs.informatica import InformaticaJob

# Step 2 - Instantiate, Authenticate and Schedule


## Step 2A - Create the object

In [3]:
t1_flow = CmJobFlow(application="ctm_python_client-Demo",sub_application="IS&T")

## Step 2B - Authenticate to Control-M

In [4]:
from ctm_python_client.session.session import Session

import getpass
ctm_uri =  "https://acb-rhctmv20.centralus.cloudapp.azure.com:8443/automation-api"
ctm_user =  "vtadinad"

if "ctm_pwd" not in locals(): # has not been enterd once, will skip next time
    ctm_pwd = getpass.getpass("Enter your Control M Password ")
ctm_pwd = "P4ssw0rd"
session = Session( endpoint=ctm_uri,username=ctm_user,password=ctm_pwd)
session.get_token()                                                                                                           

Enter your Control M Password ········


'486D4F56DBFECF1EF79A8EED27033BEF5CA5177FD6A43F524EE5612F10B5AE11EEE6F04F2A862C901749907A784A33A13FDE24B104000E04772BED02E73F435E'

In [5]:
t1_flow = CmJobFlow(application="Naga0.3",sub_application="Customer-Zero",session=session)

## Step 2C - Define the Flow Level Defaults

### Default run_ass and host

In [6]:
t1_flow.set_run_as(username="ctmuser",host="acb-rhctmv20")

### Define Default Schedule

In [7]:
# Define the schedule
monthDays = ["ALL"]
fromTime = "0001"
t1_flow.set_schedule( month_days=monthDays, from_time=fromTime)


# Step 3  - Folder

## Step 3A - Create Folder

In [8]:
# Create Folder
f1 = t1_flow.create_folder(name="SnowFlakeRightSizing")

### Step 3B - Define Variables

In [9]:
#appid is the weather api token, you obtained during setup
# variables = [ { "appid" : "481be8b9210a09cc3a3bcdcf190a3aa8"}, 
#               { "zipcode" : "68116"}, 
#               { "email" : "abaldwin@bmc.com"} ]
#t1_flow.add_variables(folder=f1, list_of_variables=variables)

# Step 4 - Create Tasks

In [10]:
start = t1_flow.add_job(f1, DummyJob(folder= f1, job_name="StartFlow"))
end = t1_flow.add_job(f1, DummyJob(folder= f1, job_name="EndFlow"))

job_params = [{"job_name":"Load_Size_Up","workflow":"wfl_warehouse_resize_up","from_time":"0001"},
              {"job_name":"Rpt_Size_Up","workflow":"wfl_rpt_warehouse_resize_up","from_time":"0300"},
              {"job_name":"UMDB_Size_Up","workflow":"wfl_umdb_warehouse_resize_up","from_time":"0500"},
              {"job_name":"Load_Size_Down","workflow":"wfl_warehouse_resize_down","from_time":"0800"},
              {"job_name":"Rpt_Size_Down","workflow":"wfl_rpt_warehouse_resize_down","from_time":"0900"},
              {"job_name":"UMDB_Size_Down","workflow":"wfl_umdb_warehouse_resize_down","from_time":"1000"},
             ]

#create jobs
jobs = {}
for j in job_params:
    job = InformaticaJob(folder=f1, job_name=j["job_name"], 
                                connection_profile = 'AWSINFAPRD',
                                repository_folder = 'Repository_Admin',
                                workflow = j["workflow"],
                                workflow_execution_mode = 'RunWholeWorkflow',
                                workflow_restart_mode = 'Recover',
                                enable_output = False,
                                enable_error_details = False,
                                priority = 'Very Low',
                                host="AWSINFAPRD", run_as="AWSINFAPRD")
    job.set_job_schedule(month_days = ["ALL"],from_time=j["from_time"] )
    jobs[j["job_name"]] = t1_flow.add_job(f1,job)

# Step 5 - Chain Tasks

In [11]:
t1_flow.chain_jobs(f1,[start, jobs["Load_Size_Up"], jobs["Load_Size_Down"], end])
t1_flow.chain_jobs(f1,[start, jobs["Rpt_Size_Up"], jobs["Rpt_Size_Down"], end])
t1_flow.chain_jobs(f1,[start, jobs["UMDB_Size_Up"], jobs["UMDB_Size_Down"], end])




['StartFlow', 'EndFlow', 'Load_Size_Up', 'Rpt_Size_Up', 'UMDB_Size_Up', 'Load_Size_Down', 'Rpt_Size_Down', 'UMDB_Size_Down']
['0', '2', '5', '1']
['StartFlow', 'EndFlow', 'Load_Size_Up', 'Rpt_Size_Up', 'UMDB_Size_Up', 'Load_Size_Down', 'Rpt_Size_Down', 'UMDB_Size_Down']
['0', '3', '6', '1']
['StartFlow', 'EndFlow', 'Load_Size_Up', 'Rpt_Size_Up', 'UMDB_Size_Up', 'Load_Size_Down', 'Rpt_Size_Down', 'UMDB_Size_Down']
['0', '4', '7', '1']


# Step 6 - Display Workflow

## Step 6A - Display DAG

In [12]:
from ctm_python_client.utils.displayDAG import DisplayDAG
try:
    d = DisplayDAG(t1_flow)
    d.display_graphviz()
except: #implies grpahviz not installed properly
    t1_flow.display()

=========== Jobflow Details ===================
Application:  Naga0.3
Sub Application:  Customer-Zero
Folder Name: SnowFlakeRightSizing 





In [15]:
d = DisplayDAG(t1_flow)
d.display_graphviz()

=========== Jobflow Details ===================
Application:  Naga0.3
Sub Application:  Customer-Zero
Folder Name: SnowFlakeRightSizing 





ExecutableNotFound: failed to execute 'dot', make sure the Graphviz executables are on your systems' PATH

## Step 6B - Display JSON

In [13]:
#t1_flow.display_json()

# Step 7 - Submit Workflow to Control-M

In [14]:
t = t1_flow.deploy()

[{'deployed_calendars': None,
 'deployed_connection_profiles': None,
 'deployed_drivers': None,
 'deployed_folders': ['SnowFlakeRightSizing'],
 'deployed_jobs': None,
 'deployment_file': 'jobs.json',
 'errors': None,
 'is_deploy_descriptor_valid': False,
 'successful_calendars_count': None,
 'successful_connection_profiles_count': 0,
 'successful_drivers_count': 0,
 'successful_folders_count': 0,
 'successful_jobs_count': 8,
 'successful_smart_folders_count': 1,
 'successful_sub_folders_count': 0,
 'warnings': None}]
Successfully submitted to Control-M
Login to None/ControlM/ and use your workflow
